In [1]:
import geopandas as gpd
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import modin.pandas as pd
import numpy as np
import shapely
from modin.config import ProgressBar
from tqdm.auto import tqdm

ProgressBar.enable()

In [2]:
tweets = pd.read_parquet("tweets-tok.parquet")
tokens = pd.read_parquet("wforms-bat.parquet")

stats = [
    [
        tweets["tokens"].map(len).sum(),
        "total",
    ],
    [
        tweets.explode("tokens")["tokens"].isin(tokens[~tokens["tt_att"]].index).sum(),
        "prefiltered",
    ],
    [
        tokens.shape[0],
        "unique",
    ],
    [
        (~tokens["tt_att"]).sum(),
        "unique prefiltered",
    ],
]

print("Tokens statistics:")
for row in stats:
    print("{: >8} {}".format(*row))

del stats

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-07-30 09:29:13,854	INFO worker.py:1636 -- Started a local Ray instance.


Estimated completion of line 1:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 2:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 6:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 10:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 18:   0%           Elapsed time: 00:00, estimated remaining time: ?

Tokens statistics:
71458267 total
 6538326 prefiltered
  925843 unique
  745121 unique prefiltered
